 # 베이지안 머신러닝

###  박태영 교수
### 연세대학교 응용통계학과

### Outline

### 6. 토픽 모델링
    6.1 전처리 과정
    6.2 단어 추출
    6.3 토픽 추출
    6.4 토픽 시각화 및 해석

#### 6.1 전처리 과정

- 필요한 모듈을 설치

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

- 네이버 쇼핑 데이터 (출처 : https://github.com/bab2min/corpus/tree/master/sentiment)
    - 수집기간: 2020년 6월 - 2020년 7월
    - 데이터 건수: 20만건
    - 데이터 내용: 제품별 평점과 후기 (긍/부정으로 분류하기 애매한 평점 3점은 제외)

In [2]:
data = pd.read_table('Naver_shopping.txt',header=None)
data.columns=['star','review']

In [3]:
data = data[:20000] # 분석시간 단축을 위해 2만건의 데이터만 선택

- 데이터를 평점별로 분리
- 같은 평점의 데이터는 점(`.`)으로 연결 후 하나의 문자열로 결합하여 문서 생성

In [4]:
doc1 = data[data.star==1].review.str.cat(sep='. ')
doc2 = data[data.star==2].review.str.cat(sep='. ')
doc3 = data[data.star==4].review.str.cat(sep='. ')
doc4 = data[data.star==5].review.str.cat(sep='. ')

- 문서의 모음 생성

In [5]:
data = pd.DataFrame([doc1,doc2,doc3,doc4],columns=['review'])
data['doc'] = data.index+1
data

,review,doc
0,주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화...,1
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고. 선물용으로 빨리 받아서 전달...,2
2,재구매 다 좋은데 하나가 이상하네요. 가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜...,3
3,배공빠르고 굿. 아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다....,4


#### (1) 한글만 남기기

- `ㄱ-ㅣ`: 자음과 모음 (ㅎㅎ, ㅋㅋ, ㅠㅠ 등)
- `가-힣`: 단어 (밥, 토끼, 선물 등)

In [6]:
data['review_filtered'] = data['review'].str.replace("[^ㄱ-ㅣ가-힣]", " ")
data

C:\Users\jaeyeon\AppData\Local\Temp\ipykernel_14004\1962974939.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['review_filtered'] = data['review'].str.replace("[^ㄱ-ㅣ가-힣]", " ")


,review,doc,review_filtered
0,주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화...,1,주문을 월 에 시켰는데 월 일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화...
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고. 선물용으로 빨리 받아서 전달...,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고 선물용으로 빨리 받아서 전달...
2,재구매 다 좋은데 하나가 이상하네요. 가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜...,3,재구매 다 좋은데 하나가 이상하네요 가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜...
3,배공빠르고 굿. 아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다....,4,배공빠르고 굿 아주좋아요 바지 정말 좋아서 개 더 구매했어요 이가격에 대박입니다 ...


#### (2) 토큰화 하기

- 불용어를 제거해주기 위해서, 의미를 가진 문자열로 "토큰화" 실행

In [7]:
doc_tokens = data['review_filtered'].apply(lambda x: x.split())

#### (2) 최소 3글자 이상의 단어만 남기기

In [8]:
doc_tokens = doc_tokens.apply(lambda x: ' '.join([w for w in x if len(w)>=3]))

#### (3) 불용어 제거하기

- 한글 불용어(`korean_stopwords.txt`) 불러오기

In [9]:
stopwords = pd.read_csv('Korean_stopwords.txt',delimiter='\t',header=None)
stopwords = list(stopwords[0].values)

- 불용어 제거하기

In [10]:
doc_tokens = doc_tokens.apply(lambda x: [item for item in x.split() if item not in stopwords])

- 문서 합치기

In [11]:
docs = doc_tokens.apply(lambda x: ' '.join(x))

#### 6.2 단어 추출

- 단어를 추출하는 방법

  (1) 형태소별로 구분하기
  
    - 빠르지만, 필요없는 단어도 함께 포함됨
    
  (2) 명사만 추출하기
  
    - 빠르지만, 용언(동사 및 형용사)이 제외됨
    
  (3) 단어와 품사를 함께 추출하기
  
    - 명사와 용언을 함께 추출이 가능하나, 느림

- 필요한 모듈 및 프로그램 설치

  (1) 한국어 자연어 처리 라이브러리인 KoNLPy는 JAVA VM 환경에서 작동되므로 반드시 JDK를 설치해야 함 

      https://www.oracle.com/java/technologies/javase-jdk15-downloads.html

  (2) OS별로 KoNLPy 사이트를 참조하여 KoNLPy 설치

      https://konlpy-ko.readthedocs.io/ko/v0.4.3/install/

In [12]:
!pip3 install konlpy

     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     - -------------------------------------- 0.6/19.4 MB 17.3 MB/s eta 0:00:02
     ---- ----------------------------------- 2.2/19.4 MB 34.3 MB/s eta 0:00:01
     --------- ------------------------------ 4.7/19.4 MB 42.7 MB/s eta 0:00:01
     ------------- -------------------------- 6.7/19.4 MB 42.9 MB/s eta 0:00:01
     --------------- ------------------------ 7.7/19.4 MB 44.7 MB/s eta 0:00:01
     ----------------- ---------------------- 8.7/19.4 MB 36.8 MB/s eta 0:00:01
     ------------------- -------------------- 9.5/19.4 MB 35.8 MB/s eta 0:00:01
     ----------------------- --------------- 11.6/19.4 MB 40.9 MB/s eta 0:00:01
     --------------------------- ----------- 13.6/19.4 MB 38.5 MB/s eta 0:00:01
     -------------------------------- ------ 16.0/19.4 MB 38.5 MB/s eta 0:00:01
     ------------------------------------ -- 18.4/19.4 MB 46.7 MB/s eta 0:00:01
     --------------------------------------  19.

- KoNLPy 설치 후, 모듈이 작동하는 파이썬 환경 확인

In [13]:
import sys
sys.executable

'C:\\Users\\jaeyeon\\anaconda3\\python.exe'

In [14]:
from konlpy.tag import Okt

- 위의 코드를 실행 시, 컴퓨터나 OS에 따라 다음과 같은 **오류**가 발생할 수 있음

  (1) `No Module named 'jpype'`, 즉 파이썬이 JAVA 라이브러리 접근 시 필요한 `JPype`가 존재하지 않는다는 오류 

      - Windows의 경우 아래의 KoNLPy 공식 사이트의 설명대로 JPype설치
      
      https://konlpy-ko.readthedocs.io/ko/v0.4.3/install/

  (2) `No module named 'konlpy'`, 즉 `konlpy`모듈을 아무 문제없이 설치했음에도 불구하고 존재하지 않는다는 오류
  
      - OS에 여러 개의 파이썬이 설치되어 있는데, 주피터 노트북에서 KoNLPy가 설치된 파이썬을 인식하지 못하므로 등록과정 필요 
      - 아래 3줄의 코드에서 #을 제거 후, 한 줄씩 코드 실행 
      - 주피터 노트북 및 콘솔창을 모두 종료 후, 다시 실행

In [15]:
# !pip install ipython # 주피터 노트북의 파이썬 커널 설치

In [16]:
# !python -m pip install ipykernel # 가상 환경에서 주피터 노트북 설치

In [17]:
# !python -m ipykernel install --user # 주피터 노트북에 파이썬 커널 등록

In [18]:
Okt().morphs('경력을 많이 쌓으려면 역경을 이겨내야 한다.') # 형태소 단위로 구분하기

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
Okt().nouns('경력을 많이 쌓으려면 역경을 이겨내야 한다.') # 명사만 추출하기

In [ ]:
Okt().pos('경력을 많이 쌓으려면 역경을 이겨내야 한다.') # 단어와 품사를 함께 추출하기

- 문서에서 명사를 추출한 뒤, 토큰화 하고, 2글자 이상인 토큰만 추출

In [19]:
docs

0    주문을 시켰는데 배송이 왔네요 ㅎㅎㅎ 회사측과는 전화도 안되고 아무런 연락을 받을수...
1    택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고 선물용으로 받아서 전달했어야 ...
2    재구매 좋은데 하나가 이상하네요 가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜린데 ...
3    배공빠르고 아주좋아요 좋아서 구매했어요 이가격에 대박입니다 바느질이 엉성하긴 하지만...
Name: review_filtered, dtype: object

In [20]:
docs['tokens'] = docs.apply(lambda x: [w for w in Okt().nouns(x) if len(w)>=2]) 
docs['tokens'].head()

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
doc1 = docs['tokens'][0]
doc2 = docs['tokens'][1]
doc3 = docs['tokens'][2]
doc4 = docs['tokens'][3]
doc_list = [doc1,doc2,doc3,doc4]

#### 6.3 토픽 추출

- 필요한 모듈을 설치

In [21]:
!pip install gensim

     ---------------------------------------- 0.0/67.1 kB ? eta -:--:--
     ---------------------------------------- 67.1/67.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20448 sha256=941da88b20ad05ebd17807dace27c66234a5e3416b60f69e777bd49b25e50463
  Stored in directory: c:\users\jaeyeon\appdata\local\pip\cache\wheels\69\f5\e5\18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3522 sha256=c6e7e3037f5fc2eb088c672a5dd358032eaa14dc18cb8136c324123ad40286a4
  Stored in directory: c:\users\jaeyeon\appdata\local\pip\cache\wheels\9d\ff\2f\afe4cd56f47de147407705626517d68bea0f3b74eb1fb168e6
Successfully built fst-pso miniful


In [22]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

- 문서에서 각 단어의 빈도수를 보여주는 단어-문서 행렬을 구하기
- 각 단어를 정수로 인코딩하고, 해당 단어의 빈도수를 구해 `corpus`에 저장
- `LdaModel`에서는 `corpus`를 입력값으로 받음

In [23]:
def LDA(doc_list,num_topics,num_words=5):
    dictionary = corpora.Dictionary(doc_list) # 단어를 단어id로 인코딩 (딕셔너리 자료형)
    corpus = [dictionary.doc2bow(text) for text in doc_list] # 단어id에 대한 빈도수 계산
    lda = LdaModel(corpus, num_topics = num_topics, id2word=dictionary, random_state=1) # LDA 
    topics = lda.print_topics(num_words=num_words) # 토픽 분포 출력
    return lda,topics,corpus,dictionary

#### (1) 토픽의 개수를 미리 지정하기  

- 토픽의 개수(`num_topics`)를 미리 지정하고, 각 토픽별 주요 단어의 개수(`num_words`) 지정

In [24]:
lda,topics,corpus,dictionary = LDA(doc_list,num_topics=3,num_words=5)

NameError: name 'doc_list' is not defined

#### (2) 토픽의 개수를 최적화하기

- 혼잡도(perplexity)를 최소화
    - 훈련용 데이터에서 얻은 토픽 모형을 테스트용 데이터에 적용하였을 때 예측을 하지 못하는 정도를 수치화
- 토픽 일관성(topic coherence)를 최대화
    - 의미론적으로 유사한 단어들이 모인 정도를 위키피디아, 구글 검색 등의 외부자료에 기반하여 수치화
    - $\log \frac{p\left(w_{i}, w_{j}\right)}{p\left(w_{i}\right) p\left(w_{j}\right)}$으로 단어들이 토픽 내에서 동시에 발생하는 경우 0보다 큰 값을 가지고, 독립인 경우 0의 값을 가지고, 대조적으로 발생하는 경우 0보다 작은 값을 가짐 
    - `coherence='c_v'` 옵션을 사용

In [ ]:
num_topics = np.arange(3,10)
num_words = 5
coherence_per_topic = []

for i in num_topics:
    lda,topics,corpus,dictionary = LDA(doc_list,num_topics=i,num_words=num_words)
    cm = CoherenceModel(model=lda, texts=doc_list, corpus=corpus, coherence='c_v') 
    coherence = cm.get_coherence()
    coherence_per_topic.append(coherence)

In [ ]:
plt.plot(num_topics,coherence_per_topic)
plt.show()

- 토픽 일관성이 토픽의 개수가 8일때 가장 크나, 차이는 크지 않고 해석의 편의성을 위해 3개의 토픽을 사용

In [ ]:
lda,topics,corpus,dictionary = LDA(doc_list,num_topics=3,num_words=5)

#### 6.4 토픽 시각화 및 해석

- 필요한 라이브러리를 설치

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

def topic_per_doc(model,corpus,topics,doc_idx):
    warnings.filterwarnings(action='ignore')
    print("==== 문서{}의 토픽 분포 ".format(doc_idx+1) + "="*69)
    for x in model[corpus[doc_idx]]:
        print('| 토픽{}의 비율: {:.3f} | {}'.format(x[0]+1,x[1],topics[x[0]][1])) # 소수점 3째짜리까지 표시
    print("="*88)

In [ ]:
topic_per_doc(lda,corpus,topics,0) # 평점이 1인 후기
topic_per_doc(lda,corpus,topics,1) # 평점이 2인 후기
topic_per_doc(lda,corpus,topics,2) # 평점이 4인 후기
topic_per_doc(lda,corpus,topics,3) # 평점이 5인 후기

- 평점이 높아질수록 **토픽1**의 비율은 높아지고, **토픽2,3**의 비율은 낮아짐
- 각 토픽이 의미하는 것을 해석하기 위해 시각화 방법을 이용

- 토픽 시각화를 위해 필요한 라이브러리 설치

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim

import warnings
warnings.filterwarnings(action='ignore')

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary,sort_topics=False)
pyLDAvis.display(vis)

- **토픽1**: 평점이 높을수록 비율이 높아지는 토픽으로, "**매일**", "**애용**", "**마음**", "배송", "선물" $\Rightarrow$ **단골제품**
- **토픽2**: 평점이 낮을수록 비율이 높아지는 토픽으로, "**제품**", "**시간**", "**최악**", "가격", "반품" $\Rightarrow$ **제품의 구매과정**
- **토픽3**: 평점이 낮을수록 비율이 높아지는 토픽으로, "생각", "**색상**", "**크기**", "**포장**", "**사이즈**", "실망" $\Rightarrow$ **제품의 품질**